元の偉大なカーネル（ https://www.kaggle.com/christofhenkel/extract-image-features-from-pretrained-nn ）を日本語化しました。
あとは説明用のコードを多少追加しました。

- Take only profile picture (if existing else black)
- pad to square aspect ratio
- resize to 256


In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook

train_df = pd.read_csv('../input/train/train.csv')
img_size = 256
batch_size = 16

In [ ]:
pet_ids = train_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

## 元画像はこーんな感じ

In [ ]:
from matplotlib import pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20,10

def load_raw_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    return image

for i in range(1,10):
    plt.subplot(330+i),plt.imshow(load_raw_image("../input/train_images/", pet_ids[i]))
plt.show()

## 前処理をします

In [ ]:
from keras.applications.densenet import preprocess_input, DenseNet121

def resize_img(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return im

def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

## resizeの処理

In [ ]:
def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image) # resize
    return new_image

for i in range(1,10):
    plt.subplot(330+i),plt.imshow(load_image("../input/train_images/", pet_ids[i]))
plt.show()

## preprocess
- 転移学習するときは「preprocess_input」を入れる
- 元々imagenetで画像を学習したときに合わせた前処理がなされるらしい

In [ ]:
def load_preprocess_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image) # preprocess_input
    return new_image

for i in range(1,10):
    plt.subplot(330+i),plt.imshow(load_preprocess_image("../input/train_images/", pet_ids[i]))
print('なんか人の目ではわかりづらくなったように見えるが、これでよいらしい')
plt.show()

## ここから特徴量抽出
- このkernelではdensenet121の重みを使っている
- 他にもいろいろな重みが使えるよ https://keras.io/applications/
- include_top=Falseにすることで、モデルの全結合層（特徴を分類する層）を入れない状態のネットワークを定義できる
- 出力は1024次元だが、1/4にするようプーリングして出力

In [ ]:
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K
inp = Input((256,256,3))
backbone = DenseNet121(input_tensor = inp, include_top = False)
x = backbone.output # この時点で 8 * 8 * 1024channel
x = GlobalAveragePooling2D()(x) # 各channelの画素平均 このままでも1024列の特徴量として出力される
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x) # 256列にするために配列末尾にdimを挿入 [None,1024] → [None,1024,1]
x = AveragePooling1D(4)(x) # 1/4にするようにプーリング
out = Lambda(lambda x: x[:,:,0])(x) # [None,256,1] → [None,256]
m = Model(inp,out)

In [ ]:
m.summary()

## petの画像を読み込んで上記ネットワークで予測。重みはimagenetで既に訓練されたもの

In [ ]:
features = {}
batch_pets = pet_ids[0:9]
batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
for i,pet_id in enumerate(batch_pets):
    try:
        batch_images[i] = load_preprocess_image("../input/train_images/", pet_id)
    except:
        pass
batch_preds = m.predict(batch_images)
for i,pet_id in enumerate(batch_pets):
    features[pet_id] = batch_preds[i]

## 無事に画像から特徴が抽出できました！
- あとは煮るなり焼くなりお好きにどうぞ
- note:今回のコンペは学習済みのimagenetのデータセット( http://starpentagon.net/analytics/ilsvrc2012_class_image/ )に犬や猫の画像が大量に入っているので特徴をちゃんと捉えた特徴量になっているのでは、と推測します。もし、imagenetで全然使用されていないカテゴリの画像から特徴量を抽出したい場合は、別途そのカテゴリの画像を用意してfine tuningしたモデルを新たに作った方が良い特徴量が抽出できると思います。

In [ ]:
train_feats = pd.DataFrame.from_dict(features, orient='index')
train_feats

## Extra

で、画像の特徴量入れるとどう精度が変わるの？

まず、抽出した特徴量を入れずに、属性情報+sentiment+text、lgbでモデリングしたkernel(private:0.382）
https://www.kaggle.com/wrosinski/baselinemodeling

上記に抽出した特徴量（+αで画像のタテヨコとかの特徴量とか）も加えてxgbでモデリングしたkernel
https://www.kaggle.com/ranjoranjan/single-xgboost-model (private:0.453）

かなり精度が違う！